# Database Relazionali

## Le specifiche 'Python Database API'

- Uniforma le implementazioni di accesso alle basi dati
- Pubblicata nel 2001 ed ampiamente adottata
- MySQLdb, cx_Oracle, Psycopg, sqlite3, ...

PEP 249 -- Python Database API Specification v2.0:
https://www.python.org/dev/peps/pep-0249/ 

### Database API - Panoramica

![DB-API diagram](../images/db-api.jpg)

### connect()

- I parametri sono totalmente dipendenti dall'implementazione
- Restituisce un oggetto di tipo [Connection][1]

[1]: https://www.python.org/dev/peps/pep-0249/#connection

In [1]:
import sqlite3
conn = sqlite3.connect(":memory:")

### L'oggetto Connection

Principali metodi:

- __cursor()__: ottiene un oggetto di tipo _Cursor_
- __commit()__: esegue la commit della transazione corrente
- __rollback()__: effettua un rollback della transazione corrente
- __close()__: chiude la connessione

In [2]:
conn

### L'oggetto Cursor

In [3]:
cursor = conn.cursor()

Tramite un cursore è possibile eseguire query e raccoglierne i risultati.

Principali metodi:

- __execute(operation[, parameters])__:
- __fetchone()__:
- __fetchall()__:
- __close()__:

### Cursor - Esecuzione di una query

Tramite il metodo **execute()** è possibile eseguire una singola query verso il database:

In [4]:
cursor.execute("CREATE TABLE countries (name VARCHAR PRIMARY KEY, code VARCHAR NOT NULL)")

Il metodo non restituisce alcun valore, ma in caso di errore verrà sollevata un'eccezione di tipo **DatabaseError** o una sua sottoclasse più specifica.

### Cursor - Esecuzione multipla di una query

Utilizzando il metodo **executemany()** è possibile eseguire con una sola istruzione più query con parametri diversi:

In [5]:
countries = [
    ("Italy", "IT"),
    ("Spain", "ES"),
    ("Germany", "DE")
]

cursor.executemany("INSERT INTO countries (name, code) VALUES (?, ?)", countries)
conn.commit()

### Cursor - Query parametriche e sicurezza

Attenzione a non confondere queste due istruzioni:

- Corretta:

In [16]:
cursor.execute("INSERT INTO countries (name, code) VALUES (?, ?)", ("France", "FR"))

- Sbagliata:

In [19]:
cursor.execute("INSERT INTO countries (name, code) VALUES ('%s', '%s')" % ("Denmark", "DK"))

### Connection - Stato della transazione e commit

- **in_transaction**: attributo booleano, _True_ se una transazione è in corso
- **commit()**: esegue la commit delle modifiche locali

In [6]:
conn.in_transaction

False

In [7]:
conn.commit()

In [8]:
conn.in_transaction

False

### Cursor - Ottenere il risultato di una query

Eseguita una query è possibile ottenere il risultato attraverso tre metodi:

- **fetchone()**: restituisce la prossima riga del resultset, e None quando non ve ne sono più
- **fetchmany([size=cursor.arraysize])**: restituisce fino a _size_ righe per volta
- **fetchall()**: restituisce l'intero risultato

### Cursor - Il metodo fetchone()

In [9]:
cursor.execute("SELECT name, code FROM countries WHERE name = :name", {"name": "Italy"})

In [10]:
record = cursor.fetchone()
record

('Italy', 'IT')

In [11]:
record = cursor.fetchone()
record

### Cursor - Il metodo fetchmany()

In [12]:
cursor.execute("SELECT name, code FROM countries")

In [13]:
cursor.fetchmany(2)

[('Italy', 'IT'), ('Spain', 'ES')]

In [14]:
cursor.fetchmany(2)

[('Germany', 'DE')]

In [15]:
cursor.fetchmany(2)

[]

### Cursor - Il metodo fetchall()

In [91]:
cursor.execute("SELECT name, code FROM countries")

In [92]:
cursor.fetchall()

[('Italy', 'IT'), ('Spain', 'ES'), ('Germany', 'DE')]

### Chiudere la connessione

Attraverso il metodo **close()** dell'oggetto Connection è possibile chiudere una connessione:

In [93]:
conn.close()

Eventuali operazioni successive su di una connessione terminata solleveranno un'eccezione:

In [94]:
conn.cursor()

ProgrammingError: Cannot operate on a closed database.

## Connettersi a MySQL con DB-API

Esistono varie librerie per MySQL che aderiscono alle specifiche DB-API.

Qui utilizzeromo **pymysql**, una implementazione _pure python_ senza alcuna dipendenza.

In [95]:
import pymysql
conn = pymysql.connect(
    host="localhost",
    port=3306,
    user="python-intro",
    password="woo8oHeiwo5i",
    database="sandbox")

### Abilitare l'autocommit delle transazioni

I driver per MySQL permettono di gestire l'autocommit delle transazioni su base connessione.

- Abilitare l'autocommit avrà l'effetto di effettuare un **commit()** implicito dopo ogni **execute()**.
- Impossibile raggruppare più query in una sola transazione
- Impossibile annullare una o più operazioni svolte nella sessione corrente

Di default non è abilitato:

In [96]:
conn.autocommit_mode

False

### Abilitare l'autocommit delle transazioni

Per abilitarlo è sufficiente passare il parametro _True_ al metodo **autocommit()** della connessione:

In [97]:
conn.autocommit(True)

In [98]:
conn.autocommit_mode

True

### Utilizzare DictCursor al posto di BaseCursor

Abbiamo visto che l'oggetto **Cursor** di default restituisce i risultati delle query sotto forma di sequenze (tuple) o sequenze di sequenza (liste di tuple).

Dicendo alla nostra istanza di **Connection** di utilizzare **DictCursor** al posto dell'implementazione di default invece potremo ottenere dei dizionari:

In [99]:
cursor = conn.cursor(pymysql.cursors.DictCursor)

In [100]:
cursor.execute("SELECT * FROM information_schema.schemata")
cursor.fetchone()

{'CATALOG_NAME': 'def',
 'SCHEMA_NAME': 'information_schema',
 'DEFAULT_CHARACTER_SET_NAME': 'utf8',
 'DEFAULT_COLLATION_NAME': 'utf8_general_ci',
 'SQL_PATH': None}

### Abilitare DictCursor a livello di connessione

E' anche possibile abilitarne l'uso a livello di connessione.

In questo modo il metodo **cursor()** restituirà sempre un'istanza di **DictCursor**:

In [101]:
conn = pymysql.connect(
    host="localhost",
    port=3306,
    user="python-intro",
    password="woo8oHeiwo5i",
    database="sandbox",
    cursorclass=pymysql.cursors.DictCursor)